# ASR for Farsi

### Import required libraries

In [ ]:
!pip install datasets transformers librosa torchaudio hazm num2fawords jiwer accelerate

In [2]:
import json
import time

import re
from hazm import Normalizer

import torch
import torchaudio
import torchaudio.transforms as transforms
import librosa

import numpy as np
from jiwer import wer

from datasets import load_dataset, Audio, load_metric

from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, DataCollatorWithPadding
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

### Load dataset from HuggingFace

In [3]:
from huggingface_hub import login
login()

In [4]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_6_1", "fa", split="train")
common_voice_test = load_dataset("mozilla-foundation/common_voice_6_1", "fa", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Remove Unnecessary Columns

In [5]:
columns_to_remove = ['down_votes', 'gender', 'locale', 'segment', 'up_votes', 'accent', 'age', 'client_id']

print("Train Set Columns (Before Drop):", common_voice_train.column_names)
print("Test Set Columns (Before Drop):", common_voice_test.column_names)

common_voice_train = common_voice_train.remove_columns(columns_to_remove)
common_voice_test = common_voice_test.remove_columns(columns_to_remove)

print("Train Set Columns (After Drop):", common_voice_train.column_names)
print("Test Set Columns (After Drop):", common_voice_test.column_names)

Train Set Columns (Before Drop): ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
Test Set Columns (Before Drop): ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment']
Train Set Columns (After Drop): ['path', 'audio', 'sentence']
Test Set Columns (After Drop): ['path', 'audio', 'sentence']


### Filter Audio

In [6]:
def filter_train_audio(batch):
    duration = len(batch["audio"]["array"]) / batch["audio"]["sampling_rate"]
    return 4.0 <= duration <= 6.0

def filter_test_audio(batch):
    duration = len(batch["audio"]["array"]) / batch["audio"]["sampling_rate"]
    return duration < 15.0

print("Common Voice Train Length (Before Filter):", len(common_voice_train))
print("Common Voice Test Length (Before Filter):", len(common_voice_test))

common_voice_train = common_voice_train.filter(filter_train_audio)
common_voice_test = common_voice_test.filter(filter_test_audio)

print("Common Voice Train Length (After Filter):", len(common_voice_train))
print("Common Voice Test Length (After Filter):", len(common_voice_test))

Common Voice Train Length (Before Filter): 7593
Common Voice Test Length (Before Filter): 5213
Common Voice Train Length (After Filter): 2217
Common Voice Test Length (After Filter): 5212


### Preprocessing

In [7]:
chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�", "&",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?",
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "=", "–", "…", "_", "”", '“', '„',
    'ā', 'š'
]


chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    # "ها": "  ها", "ئ": "ی",
    "۱۴ام": "۱۴ ام",

    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}

In [8]:
normalizer = Normalizer()

def preprocess_text(text):
    text = normalizer.normalize(text)
    text = text.lower()
    escaped_chars_to_ignore = [re.escape(char) for char in chars_to_ignore]
    text = re.sub(f"[{''.join(escaped_chars_to_ignore)}]", '', text)
    for old, new in chars_to_mapping.items():
        text = text.replace(old, new)
    return text

common_voice_train = common_voice_train.map(lambda batch: {"sentence": preprocess_text(batch["sentence"])})
common_voice_test = common_voice_test.map(lambda batch: {"sentence": preprocess_text(batch["sentence"])})

In [9]:
for i in range(3):
  print("Data", i)
  print(common_voice_train[i]["sentence"])
  print(common_voice_test[i]["sentence"])

Data 0
چه جوری آخه برانکو با دست خالی تیمشو برد فینال
از هم جداشدن خیلی سخته
Data 1
اون میوه هات رو بردار
بله مطمئن باشید هستم
Data 2
 خوبه که جامعه پزشکی
تقریبا صدو پنجاه گز دورتر از جاده


### Create Character Dictionary

In [10]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

vocab_dict["|"] = vocab_dict.pop(" ")
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict["<s>"] = len(vocab_dict)
vocab_dict["</s>"] = len(vocab_dict)

Map:   0%|          | 0/2217 [00:00<?, ? examples/s]

Map:   0%|          | 0/5212 [00:00<?, ? examples/s]

In [11]:
print("Dictionary Length:", len(vocab_dict))
print(vocab_dict)

Dictionary Length: 40
{'ش': 0, 'خ': 1, 'ک': 2, 'غ': 3, 'ن': 4, 'ء': 5, 'ط': 6, 'ظ': 7, 'د': 8, 'ر': 9, 'ا': 10, 'ح': 11, 'ذ': 12, 'ب': 13, 'چ': 14, 'ج': 16, 'س': 17, 'ئ': 18, 'و': 19, 'ت': 20, 'ژ': 21, 'ز': 22, 'ی': 23, 'پ': 24, 'ل': 25, 'ه': 26, 'م': 27, 'ق': 28, 'ض': 29, 'ف': 30, 'ع': 31, 'ص': 32, 'گ': 33, 'ث': 34, 'آ': 35, '|': 15, '[UNK]': 36, '[PAD]': 37, '<s>': 38, '</s>': 39}


### Save unique characters in JSON

In [12]:
vocab_file = "vocab.json"
with open(vocab_file, 'w') as vf:
    json.dump(vocab_dict, vf)

### Resample audio to 16kHz

In [13]:
def resample_audio(batch):
    waveform = torch.tensor(batch["audio"]["array"], dtype=torch.float32).unsqueeze(0)
    resampled_waveform = transforms.Resample(orig_freq=48000, new_freq=16000)(waveform)
    batch["audio"]["array"] = resampled_waveform.squeeze(0).numpy()
    batch["audio"]["sampling_rate"] = 16000
    return batch

def resample_audio_librosa(batch):
    y, sr = librosa.load(batch["audio"]["path"], sr=48000)
    y_resampled = librosa.resample(y, orig_sr=48000, target_sr=16000)
    batch["audio"]["array"] = y_resampled
    batch["audio"]["sampling_rate"] = 16000
    return batch


common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=48000))
common_voice_train = common_voice_train.map(resample_audio_librosa)
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=48000))
common_voice_test = common_voice_test.map(resample_audio_librosa)

### Tokenization and Feature Extraction -> Processor

---



In [14]:
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_file=vocab_file,
    unk_token="[UNK]",
    pad_token="[PAD]",
    bos_token="<s>",
    eos_token="</s>",
    word_delimiter_token="|"
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Data Collator

In [15]:
def prepare_dataset(batch):
    audio = batch["audio"]["array"]
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")

    with processor.as_target_processor():
        labels = processor(batch["sentence"], return_tensors="pt")

    return {
        "input_values": inputs.input_values,
        "labels": labels.input_ids
    }

common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/2217 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5212 [00:00<?, ? examples/s]

In [16]:
# from dataclasses import dataclass, field
# from typing import Any, Dict, List, Union

# @dataclass
# class DataCollatorCTCWithPadding:
#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True

#     def __call__(self, features: List[Dict[str, Union[List[int], np.ndarray, Dict[str, np.ndarray]]]]) -> Dict[str, torch.Tensor]:
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 return_tensors="pt",
#             )

#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

# data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=processor.tokenizer, padding=True)

### Evaluation Metric

In [18]:
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    labels_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=labels_str)
    return {"wer": wer}

<ipython-input-18-42c83d9cc2c5>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


### Load Model

In [19]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53",
                                       gradient_checkpointing=True,
                                       ctc_loss_reduction="mean",
                                       pad_token_id=processor.tokenizer.pad_token_id)

model.freeze_feature_extractor()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2274: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [20]:
!pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [21]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-large-xlsr-persian-demo",
    group_by_length=True,
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=5,
    fp16=True,
    gradient_checkpointing=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    warmup_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
start_time = time.time()
trainer.train()
end_time = time.time()
train_time = end_time - start_time
print(f"Training time: {train_time} seconds")

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['input_values', 'labels']

In [23]:
print("Trainable weights:")
total_trainable_params = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        num_params = param.numel()
        print(f"{name}: {num_params}")
        total_trainable_params += num_params

print(f"Total Trainable Parameters: {total_trainable_params}")

Trainable weights:
wav2vec2.masked_spec_embed: 1024
wav2vec2.feature_projection.layer_norm.weight: 512
wav2vec2.feature_projection.layer_norm.bias: 512
wav2vec2.feature_projection.projection.weight: 524288
wav2vec2.feature_projection.projection.bias: 1024
wav2vec2.encoder.pos_conv_embed.conv.bias: 1024
wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0: 128
wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1: 8388608
wav2vec2.encoder.layer_norm.weight: 1024
wav2vec2.encoder.layer_norm.bias: 1024
wav2vec2.encoder.layers.0.attention.k_proj.weight: 1048576
wav2vec2.encoder.layers.0.attention.k_proj.bias: 1024
wav2vec2.encoder.layers.0.attention.v_proj.weight: 1048576
wav2vec2.encoder.layers.0.attention.v_proj.bias: 1024
wav2vec2.encoder.layers.0.attention.q_proj.weight: 1048576
wav2vec2.encoder.layers.0.attention.q_proj.bias: 1024
wav2vec2.encoder.layers.0.attention.out_proj.weight: 1048576
wav2vec2.encoder.layers.0.attention.out_proj.bias: 1024
wav2